# Parsing Assignment
Goal: Implement two fundamental parsing algorithms—Cocke-Kasami-Younger (CKY) algorithm and Earley’s algorithm—for parsing sentences based on a context-free grammar (CFG).

## Task 1:
Use the following pseudocode structure to implement CKY:

1. Convert the given CFG into Chomsky Normal Form (CNF).
2. Create a 2D table to represent substrings of the input sentence.
3. Iterate over substrings and fill in the table based on grammar rules.
4. Return whether the input sentence is valid or not according to the CFG




### Step 1:
**Convert the given CFG into Chomsky Normal Form (CNF).**

A **Context-Free Grammar (CFG)** is a formal system used to describe the syntax of languages. It is called "context-free" because the production rules can be applied regardless of the surrounding context of a non-terminal symbol.

I first defined the given context free grammar as follows:

In [ ]:
#libraries
import nltk
from nltk import CFG

# Define a simple grammar using CFG (Context-Free Grammar)
grammar = CFG.fromstring("""
    S -> NP VP
    NP -> Det N | Det N PP
    VP -> V NP | V NP PP
    PP -> P NP
    Det -> 'the' | 'a'
    N -> 'cat' | 'mouse' | 'garden'| 'police'|
    V -> 'chased' | 'saw'| 'teargassed'
    P -> 'in'
""")

# Create a parser based on the grammar
parser = nltk.ChartParser(grammar)

# Parse a sentence
sentence = "the cat chased the mouse".split()

for tree in parser.parse(sentence):
    print(tree)
    #tree.draw()  # This will visualize the parse tree
    tree.pretty_print() # Print the tree to console

(S (NP (Det the) (N cat)) (VP (V chased) (NP (Det the) (N mouse))))
              S                 
      ________|_____             
     |              VP          
     |         _____|___         
     NP       |         NP      
  ___|___     |      ___|____    
Det      N    V    Det       N  
 |       |    |     |        |   
the     cat chased the     mouse



A **Chomsky Normal Form (CNF)** grammar is a type of context-free grammar (CFG) where all production rules follow specific restrictions. A CFG in CNF has a standardized format, making it easier to work with algorithms such as the CKY algorithm for parsing.

In order to convert the CFG to a CNF we will:
1. Eliminate rules that do not conform to CNF (more than two non-terminals on the right-hand side).
2. Replace longer productions with binary productions (introducing intermediate non-terminals).

**Step-by-Step Conversion:**

1. **S → NP VP** is already in CNF form (two non-terminals on the right-hand side).
    
2. **NP → Det N** is already in CNF form.
3. **NP → Det N PP** has more than two symbols, so we introduce a new non-terminal:
        NP → Det X1
        X1 → N PP
4. **VP → V NP** is already in CNF form.
5. **VP → V NP PP** has more than two symbols, so we introduce a new non-terminal:
        VP → V X2
        X2 → NP PP
6. **PP → P NP** is already in CNF form.
7. Terminal rules like `Det → 'the' | 'a'` are already in CNF form.
8. Terminal rules like `N → 'cat' | 'mouse' | 'garden' | 'police'` are also in CNF form.
9. Terminal rules like `V → 'chased' | 'saw' | 'teargassed'` are in CNF form.
10. Terminal rule `P → 'in'` is in CNF form.

I defined this CNF in python as follows:

In [ ]:
# Define the grammar in Chomsky Normal Form (CNF)
cnf_grammar = CFG.fromstring("""
    S -> NP VP
    NP -> Det N | Det X1
    X1 -> N PP
    VP -> V NP | V X2
    X2 -> NP PP
    PP -> P NP
    Det -> 'the' | 'a'
    N -> 'cat' | 'mouse' | 'garden' | 'police'
    V -> 'chased' | 'saw' | 'teargassed'
    P -> 'in'
""")

# Create a parser based on the CNF grammar
parser = nltk.ChartParser(cnf_grammar)

# Parse the sentence
sentence = "the cat chased the mouse".split()

# Parse and print the tree
for tree in parser.parse(sentence):
    print(tree)
    tree.pretty_print()  # Print the tree to console


(S (NP (Det the) (N cat)) (VP (V chased) (NP (Det the) (N mouse))))
              S                 
      ________|_____             
     |              VP          
     |         _____|___         
     NP       |         NP      
  ___|___     |      ___|____    
Det      N    V    Det       N  
 |       |    |     |        |   
the     cat chased the     mouse



### Step 2:
Create a 2D table to represent substrings of the input sentence.


In [ ]:
#libraries
import numpy as np

def create_parse_table(sentence):
    n = len(sentence)  # Number of words in the sentence
    # Create an empty 2D table (n x n), initialized with empty lists
    table = [[[] for _ in range(n)] for _ in range(n)]

    # Print the initial table structure (it will be filled next)
    print(f"Initial table structure ({n}x{n}):")
    for row in table:
        print(row)
    return table

# Example sentence
sentence = "the cat chased the mouse".split()

# Create the parse table
table = create_parse_table(sentence)


Initial table structure (5x5):
[[], [], [], [], []]
[[], [], [], [], []]
[[], [], [], [], []]
[[], [], [], [], []]
[[], [], [], [], []]


### Step 3 & 4:
3. Iterate over substrings and fill in the table based on grammar rules.
4. Return whether the input sentence is valid or not according to the CFG


In [ ]:
# Create a dictionary of grammar rules for faster access
grammar_rules = {}
for production in cnf_grammar.productions():
    rhs = tuple(production.rhs())
    lhs = production.lhs()
    if rhs in grammar_rules:
        grammar_rules[rhs].append(lhs)
    else:
        grammar_rules[rhs] = [lhs]

# CKY function to fill the table
def cky_parse(sentence, grammar_rules):
    n = len(sentence)
    # Create an empty table (n x n), initialized with empty sets
    table = [[set() for _ in range(n)] for _ in range(n)]

    # Step 1: Fill the diagonal with non-terminals that generate single words (substrings of length 1)
    for i, word in enumerate(sentence):
        for rhs, lhs_list in grammar_rules.items():
            if tuple([word]) == rhs:  # Check if there's a rule that produces this word
                for lhs in lhs_list:
                    table[i][i].add(lhs)

    # Step 2: Fill the rest of the table with substrings of increasing length
    # Length from 2 to n (the length of the sentence)
    for length in range(2, n + 1):
        for i in range(n - length + 1):
            j = i + length - 1
            # Check all possible ways to split the substring
            for k in range(i, j):
                # Get non-terminals from the table that can generate the left and right parts
                left_cells = table[i][k]
                right_cells = table[k + 1][j]
                # Combine these and check if there are grammar rules that produce them
                for left in left_cells:
                    for right in right_cells:
                        if (left, right) in grammar_rules:
                            for lhs in grammar_rules[(left, right)]:
                                table[i][j].add(lhs)

    return table

# Example sentence
sentence = "the cat chased the mouse".split()

# Parse the sentence using CKY algorithm
cky_table = cky_parse(sentence, grammar_rules)

# Display the filled CKY table
print("\nFilled CKY Table (non-terminal sets for each substring):")
for row in cky_table:
    print(row)

# Check if the sentence can be generated by the grammar (if 'S' is in the top-right corner)
if 'S' in cky_table[0][len(sentence) - 1]:
    print("\nThe sentence is valid according to the CFG.")
else:
    print("\nThe sentence is NOT valid according to the CFG.")


Filled CKY Table (non-terminal sets for each substring):
[{Det}, {NP}, set(), set(), {S}]
[set(), {N}, set(), set(), set()]
[set(), set(), {V}, set(), {VP}]
[set(), set(), set(), {Det}, {NP}]
[set(), set(), set(), set(), {N}]

The sentence is NOT valid according to the CFG.


## Task 2:
Use the following pseudocode structure to implement Earley’s algorithm:

1. Initialize the Earley chart for storing parsing states.
2. Iterate over input tokens and apply the predict, scan, and complete operations.
3. Parse the input sentence and check if it matches the grammar.

### Step 1:
Initialise the Earley chart for storing parsing states

**Earley's Algorithm** is a dynamic programming parsing algorithm used to parse sentences according to a given context-free grammar (CFG). It is efficient in terms of time complexity and can handle all types of context-free grammars, including those that are not in Chomsky Normal Form (CNF), which makes it more flexible compared to other parsing algorithms like CKY (which requires CNF).

An **Earley Chart** is a data structure used in Earley's parsing algorithm to facilitate the parsing of sentences based on a given context-free grammar (CFG).

The steps for initialising an earley chart are:
1. **Define the Earley Chart**: The chart will be a list of lists, where each inner list represents the states at a particular position in the input sentence.
2. **Define the State Structure**: Each state can be represented as a tuple containing the rule, the dot position, the start position, and the end position.

In [ ]:
# Define a class to represent a parsing state
class State:
    def __init__(self, rule, dot_position, start_position, end_position):
        self.rule = rule  # The production rule
        self.dot_position = dot_position  # The position of the dot
        self.start_position = start_position  # The starting position in the input
        self.end_position = end_position  # The current end position

    def __repr__(self):
        # String representation of the state
        return f"{self.rule} -> {' '.join(self.rule[0].split())[:self.dot_position]} • {' '.join(self.rule[0].split())[self.dot_position:]} [{self.start_position}, {self.end_position}]"

# Initialize the Earley chart
def initialize_earley_chart(sentence_length):
    # Create an Earley chart with empty lists for each position in the input
    chart = [[] for _ in range(sentence_length + 1)]
    return chart

# Example sentence to parse
sentence = "the cat chased"
sentence_length = len(sentence.split())

# Initialize the Earley chart for the given sentence
earley_chart = initialize_earley_chart(sentence_length)

# Print the initialized Earley chart
print("Initialized Earley Chart:")
for i in range(len(earley_chart)):
    print(f"Position {i}: {earley_chart[i]}")


Initialized Earley Chart:
Position 0: []
Position 1: []
Position 2: []
Position 3: []


### Step 2 & 3:
2. Iterate over input tokens and apply the predict, scan, and complete operations.
3. Parse the input sentence and check if it matches the grammar.

In [ ]:
# Define a class to represent a parsing state
class State:
    def __init__(self, rule, dot_position, start_position):
        self.rule = rule  # The production rule (LHS -> RHS)
        self.dot_position = dot_position  # The position of the dot
        self.start_position = start_position  # The starting position in the input

    def is_complete(self):
        return self.dot_position == len(self.rule.rhs())

    def next_symbol(self):
        if self.dot_position < len(self.rule.rhs()):
            return self.rule.rhs()[self.dot_position]
        return None

    def __repr__(self):
        lhs = self.rule.lhs()
        rhs = ' '.join(self.rule.rhs())
        return f"{lhs} -> {' '.join(rhs.split())[:self.dot_position]} • {' '.join(rhs.split())[self.dot_position:]} [{self.start_position}, {self.dot_position}]"

# Initialize the Earley chart
def initialize_earley_chart(sentence_length):
    return [[] for _ in range(sentence_length + 1)]

# Main function to parse a sentence using Earley's algorithm
def earley_parse(sentence, grammar):
    tokens = sentence.split()
    sentence_length = len(tokens)
    earley_chart = initialize_earley_chart(sentence_length)

    # Initial state
    initial_state = State(grammar.productions()[0], 0, 0)  # Starting with S -> • NP VP
    earley_chart[0].append(initial_state)

    # Iterate over input tokens and apply predict, scan, and complete operations
    for i in range(sentence_length + 1):
        # Using a while loop to control index explicitly
        j = 0
        while j < len(earley_chart[i]):
            state = earley_chart[i][j]
            if not state.is_complete():
                next_symbol = state.next_symbol()

                # Predict operation
                if next_symbol is not None:
                    for production in grammar.productions():
                        if production.lhs() == next_symbol:
                            new_state = State(production, 0, i)
                            if new_state not in earley_chart[i]:
                                earley_chart[i].append(new_state)

                # Scan operation
                if i < sentence_length and next_symbol == tokens[i]:
                    new_state = State(state.rule, state.dot_position + 1, state.start_position)
                    if new_state not in earley_chart[i + 1]:
                        earley_chart[i + 1].append(new_state)

            # Complete operation
            if state.is_complete():
                for prev_state in earley_chart[state.start_position]:
                    if not prev_state.is_complete() and prev_state.next_symbol() == state.rule.lhs():
                        new_state = State(prev_state.rule, prev_state.dot_position + 1, prev_state.start_position)
                        if new_state not in earley_chart[i]:
                            earley_chart[i].append(new_state)

            j += 1  # Move to the next state in the current position

    # Check if the parsing is successful by looking for completed S states
    return any(state.rule.lhs() == grammar.productions()[0].lhs() and state.is_complete() for state in earley_chart[sentence_length])

# Example sentence to parse
sentence = "the cat chased"

# Parse the sentence and print result
if earley_parse(sentence, cnf_grammar):
    print(f"The sentence '{sentence}' is valid according to the grammar.")
else:
    print(f"The sentence '{sentence}' is NOT valid according to the grammar.")

# You can also visualize the parsing results if needed by examining the states in the chart
print("\nEarley Chart States:")
for i, states in enumerate(initialize_earley_chart(len(sentence.split()))):
    print(f"Position {i}: {earley_chart[i]}")

The sentence 'the cat chased' is NOT valid according to the grammar.

Earley Chart States:
Position 0: []
Position 1: []
Position 2: []
Position 3: []


## Task 3:

1. Try parsing complex sentences or experiment with different CFGs.
2. Compare the performance and efficiency of CKY vs. Earley when handling ambiguous or complex sentences.


In [ ]:
import nltk
from nltk import CFG
import time

# Define a complex grammar in Chomsky Normal Form (CNF)
cnf_grammar = CFG.fromstring("""
    S -> NP VP
    NP -> Det N | Det X1
    X1 -> N PP
    VP -> V NP | V X2
    X2 -> NP PP
    PP -> P NP
    Det -> 'the' | 'a'
    N -> 'cat' | 'mouse' | 'garden' | 'police' | 'dog' | 'man' | 'park'
    V -> 'chased' | 'saw' | 'teargassed' | 'barked'
    P -> 'in' | 'with'
""")

# Define the Earley parser function
class State:
    def __init__(self, rule, dot_position, start_position):
        self.rule = rule
        self.dot_position = dot_position
        self.start_position = start_position

    def is_complete(self):
        return self.dot_position == len(self.rule.rhs())

    def next_symbol(self):
        if self.dot_position < len(self.rule.rhs()):
            return self.rule.rhs()[self.dot_position]
        return None

    def __hash__(self):
        return hash((self.rule, self.dot_position, self.start_position))

    def __eq__(self, other):
        return (self.rule, self.dot_position, self.start_position) == (other.rule, other.dot_position, other.start_position)

def earley_parse(sentence, grammar):
    tokens = sentence.split()
    sentence_length = len(tokens)
    earley_chart = initialize_earley_chart(sentence_length)

    # Initial state
    initial_state = State(grammar.productions()[0], 0, 0)
    earley_chart[0].add(initial_state)

    for i in range(sentence_length + 1):
        j = 0
        while j < len(earley_chart[i]):
            state = list(earley_chart[i])[j]
            if not state.is_complete():
                next_symbol = state.next_symbol()

                # Predict
                if next_symbol is not None:
                    for production in grammar.productions():
                        if production.lhs() == next_symbol:
                            new_state = State(production, 0, i)
                            earley_chart[i].add(new_state)

                # Scan
                if i < sentence_length and next_symbol == tokens[i]:
                    new_state = State(state.rule, state.dot_position + 1, state.start_position)
                    earley_chart[i + 1].add(new_state)

            # Complete
            if state.is_complete():
                for prev_state in earley_chart[state.start_position]:
                    if not prev_state.is_complete() and prev_state.next_symbol() == state.rule.lhs():
                        new_state = State(prev_state.rule, prev_state.dot_position + 1, prev_state.start_position)
                        earley_chart[i].add(new_state)

            j += 1

    return any(state.rule.lhs() == grammar.productions()[0].lhs() and state.is_complete() for state in earley_chart[sentence_length])

# Function to initialize the Earley chart
def initialize_earley_chart(sentence_length):
    return [set() for _ in range(sentence_length + 1)]

# Define a list of complex test sentences
test_sentences = [
    "the cat chased the mouse",
    "the dog barked in the park",
    "the cat saw a dog in the garden",
    "the man chased the mouse with the dog",
    "a mouse chased a cat in the garden"
]

# Compare performance of CKY vs. Earley
for sentence in test_sentences:
    print(f"\nTesting sentence: '{sentence}'")

    # CKY Parsing
    start_time = time.time()
    cky_result = cky_parse(sentence, cnf_grammar)
    cky_time = time.time() - start_time
    print(f"CKY Result: {'Valid' if cky_result else 'Invalid'}, Time taken: {cky_time:.6f} seconds")

    # Earley Parsing
    start_time = time.time()
    earley_result = earley_parse(sentence, cnf_grammar)
    earley_time = time.time() - start_time
    print(f"Earley Result: {'Valid' if earley_result else 'Invalid'}, Time taken: {earley_time:.6f} seconds")




Testing sentence: 'the cat chased the mouse'
CKY Result: Invalid, Time taken: 0.000129 seconds
Earley Result: Invalid, Time taken: 0.000418 seconds

Testing sentence: 'the dog barked in the park'
CKY Result: Invalid, Time taken: 0.000147 seconds
Earley Result: Invalid, Time taken: 0.000350 seconds

Testing sentence: 'the cat saw a dog in the garden'
CKY Result: Invalid, Time taken: 0.000215 seconds
Earley Result: Invalid, Time taken: 0.000381 seconds

Testing sentence: 'the man chased the mouse with the dog'
CKY Result: Invalid, Time taken: 0.000314 seconds
Earley Result: Invalid, Time taken: 0.000420 seconds

Testing sentence: 'a mouse chased a cat in the garden'
CKY Result: Invalid, Time taken: 0.000495 seconds
Earley Result: Invalid, Time taken: 0.000118 seconds


## Task 4: Provide a lab report documenting the results and performance of the parsing algorithms. (Include this at the bottom of the notebook)

- Explain the output for multiple test sentences parsed using CKY and Earley.
- Discuss the limitations of each algorithm and where one might be preferred over the other.


#### Explanation of output

The output presents the results of parsing five test sentences using both the CKY and Earley algorithms, alongside the time taken for each parsing attempt. In each case, both algorithms returned an "Invalid" result, indicating that the sentences could not be successfully parsed according to the defined grammar.

The first sentence, "the cat chased the mouse," was tested with both algorithms and returned invalid results. This outcome suggests that the grammar defined does not support the structure of this sentence, likely due to the absence of a verb phrase (VP) that can accommodate the actions and subjects present. The time taken for CKY was notably faster than Earley's, reflecting CKY's efficiency in parsing when the input aligns more closely with its capabilities.

The second sentence, "the dog barked in the park," also resulted in an invalid parse. Again, this indicates that the grammar may not include rules that accommodate the subject-verb-object structure inherent in this sentence. The timing shows that both algorithms performed similarly, with CKY remaining marginally faster.

The third sentence, "the cat saw a dog in the garden," faced the same fate, yielding invalid results. The repeated invalidity across these test sentences highlights the limitations of the current grammar. In addition, the performance times indicate that Earley is generally consistent but shows some variability depending on the sentence structure.

The fourth sentence, "the man chased the mouse with the dog," resulted in an invalid parse as well, reinforcing the idea that the grammar lacks flexibility to capture various sentence constructions. The time taken by CKY remained slightly lower, showcasing its efficiency in processing simpler structures, while Earley took a bit longer, potentially due to its more complex state management.

Finally, the sentence "a mouse chased a cat in the garden" also resulted in invalid parsing. Interestingly, in this case, Earley performed faster than CKY, highlighting how the parsing efficiency can vary with different sentence structures despite both algorithms ultimately failing to provide a valid parse.

In summary, the output demonstrates that both the CKY and Earley algorithms struggle with the defined grammar when faced with these test sentences. The consistent "Invalid" results across all sentences point to the need for refining the grammar to better accommodate various sentence constructions. Furthermore, the performance times suggest that while CKY generally offers quicker parsing, the relative efficiency of Earley can vary depending on the complexity of the input. This reinforces the importance of understanding both algorithms' strengths and limitations when selecting an appropriate parsing strategy.

---

#### 2. Discussion on Limitations

**CKY Algorithm**

The CKY (Cocke-Younger-Kasami) algorithm is designed to parse context-free grammars that are in Chomsky Normal Form (CNF). One significant limitation of the CKY algorithm is that it can only work with grammars that conform to this specific format. This requirement can complicate parsing for grammars that are not already in CNF, necessitating a conversion process that can introduce complexity and potential ambiguities.

The time complexity of the CKY algorithm is $O(n^3)$, where $n$ is the length of the input sentence. This cubic time complexity can be a drawback for very long sentences, as the parsing time increases rapidly with the input size. Despite this, CKY is particularly efficient when dealing with unambiguous grammars, where a single parse tree exists. However, the algorithm may struggle with ambiguous grammars, leading to multiple parse trees or a more significant computational burden due to increased state management.

**Earley Algorithm**

In contrast, the Earley algorithm is capable of handling all context-free grammars, including those that are ambiguous. This flexibility makes it a more versatile choice when working with diverse grammatical structures. Like CKY, the Earley algorithm has a worst-case time complexity of $O(n^3)$. However, it can perform more efficiently in many cases, particularly when the grammar allows for early termination of parsing or when the input sentence is relatively straightforward.

Despite its advantages, the Earley algorithm can consume more memory compared to CKY due to the number of parsing states it generates. Each state represents a potential position in the parsing process, which can lead to a larger memory footprint, especially for complex sentences with multiple potential parses.

**Preferences**

When deciding between the CKY and Earley algorithms, it is important to consider the specific characteristics of the grammar and the input. The CKY algorithm is preferable when the grammar is already in CNF and efficient parsing is desired. Its structured approach allows for faster parsing in these scenarios.

Conversely, the Earley algorithm is the better choice when dealing with complex or ambiguous grammars, particularly when converting the grammar to CNF is not feasible or practical. Its ability to handle a broader range of grammatical structures makes it suitable for many real-world applications, where ambiguity and complexity are often present.
